<a href="https://colab.research.google.com/github/Akshith213/HackOffV3---Team-DrainBrain/blob/main/Final_Chatbot_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell to import/download all the neccesary libraries and also to instantiate 'FAISS' document store  and store the medical corpus in it.
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4
!pip install torch==1.6.0+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

doc_dir = "data/healthcare_corpus"
s3_url = "https://drainbrain.s3.amazonaws.com/Hackoff+Data-20201129T103328Z-001.zip"
#s3_url = "https://drainbrain.s3.amazonaws.com/Drugs+hack+off-20201213T060446Z-001+(2).zip"

fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

document_store.write_documents(dicts)

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-3slvrlgh
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-3slvrlgh
     |████████████████████████████████| 215kB 23.4MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 51kB 10.6MB/s 
     |████████████████████████████████| 81kB 14.0MB/s 
     |████████████████████████████████| 327kB 51.1MB/s 
     |████████████████████████████████| 327kB 56.1MB/s 
     |████████████████████████████████| 92kB 14.8MB/s 
     |████████████████████████████████| 983kB 54.4MB/s 
     |████████████████████████████████| 5.5MB 55.7MB/s 
     |████████████████████████████████| 143kB 55.8MB/s 
     |████████████████████████████████| 225kB 59.0MB/s 
     |████████████████████████████████| 3.0MB 48.4MB/s 
     |████████████████████████████████| 7.2MB 30.5MB/s 
     |████████████████████████████████| 3.9MB 66.3MB/s 
     |█████████████████████████████

12/13/2020 08:16:00 - INFO - faiss -   Loading faiss with AVX2 support.
12/13/2020 08:16:00 - INFO - faiss -   Loading faiss.
12/13/2020 08:16:02 - INFO - haystack.preprocessor.utils -   Fetching from https://drainbrain.s3.amazonaws.com/Hackoff+Data-20201129T103328Z-001.zip to `data/healthcare_corpus`
100%|██████████| 13246178/13246178 [00:00<00:00, 50865540.84B/s]
12/13/2020 08:16:03 - WARNING - haystack.preprocessor.utils -   Skipped file data/healthcare_corpus/Hackoff Data/webscrap.py as type .py is not supported here. See haystack.file_converter for support of more file types
12/13/2020 08:16:03 - WARNING - haystack.preprocessor.utils -   Skipped file data/healthcare_corpus/Hackoff Data/scrap.py as type .py is not supported here. See haystack.file_converter for support of more file types
12/13/2020 08:16:03 - INFO - haystack.preprocessor.utils -   Converting data/healthcare_corpus/Hackoff Data/Thoracic aortic aneurysm.txt
12/13/2020 08:16:03 - INFO - haystack.preprocessor.utils -  

In [2]:
# Run this cell to instantiate the 'Dense Passage Retrieval' Retriever
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=True,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
document_store.update_embeddings(retriever)


12/13/2020 08:17:57 - INFO - filelock -   Lock 139657536725400 acquired on /root/.cache/torch/transformers/4b05580c0bfb2b640a50c1c6ae3fe9bca923871a29e0182927c086905d6c4c47.7652e92693c670fb8dfd7ec1f9191e3f82673742ff6a86cde9133a4ea6002ced.lock


12/13/2020 08:17:57 - INFO - filelock -   Lock 139657536725400 released on /root/.cache/torch/transformers/4b05580c0bfb2b640a50c1c6ae3fe9bca923871a29e0182927c086905d6c4c47.7652e92693c670fb8dfd7ec1f9191e3f82673742ff6a86cde9133a4ea6002ced.lock
12/13/2020 08:17:57 - INFO - filelock -   Lock 139657536725400 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


12/13/2020 08:17:57 - INFO - filelock -   Lock 139657536725400 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


12/13/2020 08:17:58 - INFO - filelock -   Lock 139654915771584 acquired on /root/.cache/torch/transformers/8fdd0d2838c23f921379f2b0322aecf406cbdaa97ffecc544e3a1d49a7c302bd.6f90756c59007364d7842118056ad653f39f4d340fbe20bcc04037d2a45cb0f7.lock


12/13/2020 08:18:04 - INFO - filelock -   Lock 139654915771584 released on /root/.cache/torch/transformers/8fdd0d2838c23f921379f2b0322aecf406cbdaa97ffecc544e3a1d49a7c302bd.6f90756c59007364d7842118056ad653f39f4d340fbe20bcc04037d2a45cb0f7.lock


12/13/2020 08:18:08 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/13/2020 08:18:08 - INFO - filelock -   Lock 139654943155424 acquired on /root/.cache/torch/transformers/f6388f32b32eac5dad8f0f9c7009ce69e967c1b65ebae62f805fced8022ea991.9500f04f28d7c0ca5f9c265db7ba5030897a2d752451412827f7dec185b1ee36.lock


12/13/2020 08:18:09 - INFO - filelock -   Lock 139654943155424 released on /root/.cache/torch/transformers/f6388f32b32eac5dad8f0f9c7009ce69e967c1b65ebae62f805fced8022ea991.9500f04f28d7c0ca5f9c265db7ba5030897a2d752451412827f7dec185b1ee36.lock


12/13/2020 08:18:09 - INFO - filelock -   Lock 139654906884048 acquired on /root/.cache/torch/transformers/d1c705617c02da7a616f4b5a8cb445a7f78e84bc4f9e26378c89901d97e16d78.232fed629becb590e5b2ac6c6124f9d1561ef7a1d17ad0394232dd46a0835002.lock


12/13/2020 08:18:15 - INFO - filelock -   Lock 139654906884048 released on /root/.cache/torch/transformers/d1c705617c02da7a616f4b5a8cb445a7f78e84bc4f9e26378c89901d97e16d78.232fed629becb590e5b2ac6c6124f9d1561ef7a1d17ad0394232dd46a0835002.lock


12/13/2020 08:18:19 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/13/2020 08:18:27 - INFO - haystack.document_store.faiss -   Updating embeddings for 4263 docs...
Creating Embeddings: 100%|██████████| 267/267 [01:04<00:00,  4.15 Batches/s]
12/13/2020 08:19:48 - INFO - haystack.document_store.faiss -   Indexing embeddings and updating vectors_ids...
100%|██████████| 1/1 [00:00<00:00,  1.17it/s]


In [3]:
#Run this cell to instantiate the reader which is a Roberta base model pretrained on Stanford Squad dataset
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

12/13/2020 08:19:53 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
12/13/2020 08:19:53 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
12/13/2020 08:19:53 - INFO - filelock -   Lock 139657521438848 acquired on /root/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.6060f348ba2b58d6d30b5324910152ffc512e7c3891ed13f22844f1a9b5c0d0f.lock


12/13/2020 08:19:53 - INFO - filelock -   Lock 139657521438848 released on /root/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.6060f348ba2b58d6d30b5324910152ffc512e7c3891ed13f22844f1a9b5c0d0f.lock


12/13/2020 08:19:53 - INFO - filelock -   Lock 139654905325832 acquired on /root/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.a851909c96149f062acca04d647da88d0dcd3a52cd5a8c7169e89fc6e5971c7b.lock


12/13/2020 08:20:00 - INFO - filelock -   Lock 139654905325832 released on /root/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.a851909c96149f062acca04d647da88d0dcd3a52cd5a8c7169e89fc6e5971c7b.lock


Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/13/2020 08:20:03 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/13/2020 08:20:07 - INFO - filelock -   Lock 139654910998568 acquired on /root/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock


12/13/2020 08:20:07 - INFO - filelock -   Lock 139654910998568 released on /root/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock
12/13/2020 08:20:07 - INFO - filelock -   Lock 139654907973584 acquired on /root/.cache/torch/transformers/b901c69e8e7da4a24c635ad81d016d274f174261f4f5c144e43f4b00e242c3b0.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock


12/13/2020 08:20:08 - INFO - filelock -   Lock 139654907973584 released on /root/.cache/torch/transformers/b901c69e8e7da4a24c635ad81d016d274f174261f4f5c144e43f4b00e242c3b0.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
12/13/2020 08:20:08 - INFO - filelock -   Lock 139654910519616 acquired on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.6e217123a3ada61145de1f20b1443a1ec9aac93492a4bd1ce6a695935f0fd97a.lock


12/13/2020 08:20:08 - INFO - filelock -   Lock 139654910519616 released on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.6e217123a3ada61145de1f20b1443a1ec9aac93492a4bd1ce6a695935f0fd97a.lock
12/13/2020 08:20:08 - INFO - filelock -   Lock 139654910313920 acquired on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.ec06af3e1b426682955dab3bd553eaf178b6eafac9079fc133925e0e2654213e.lock


12/13/2020 08:20:08 - INFO - filelock -   Lock 139654910313920 released on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.ec06af3e1b426682955dab3bd553eaf178b6eafac9079fc133925e0e2654213e.lock


12/13/2020 08:20:08 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
12/13/2020 08:20:08 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
12/13/2020 08:20:08 - INFO - farm.infer -    0 
12/13/2020 08:20:08 - INFO - farm.infer -   /w\
12/13/2020 08:20:08 - INFO - farm.infer -   /'\
12/13/2020 08:20:08 - INFO - farm.infer -   


In [4]:
# Run this cell to create a pipeline containing both retriever and reader
finder = Finder(reader, retriever)

12/13/2020 08:20:13 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.


In [5]:
# Function to output the predictions from the finder
def print_final_answers(predictions):
  pred=[]
  final_ans=''
  ans_1=''
  for i in range (len(predictions['answers'])):
    ans=predictions['answers'][i]['answer']
    con=predictions['answers'][i]['context']
    pro=predictions['answers'][i]['probability']
    #print('Answer: ',ans)
    #print('Context: ',con)
    #print('Probability: ',pro)
    #print('-------------------------------------------------------------------------------------------------------------------------------------------------------------')
    ans_1=str(ans)+';'+str(con)+';'+str(pro)+'@'
    final_ans=str(final_ans)+str(ans_1)
  return str(final_ans)

In [ ]:
# If you want to check the model performance without any help of an interface, uncomment the 3 lines of code below, run this cell and ask your medical query
#que=input()
#prediction = finder.get_answers(question=que, top_k_retriever=15, top_k_reader=3)
#print_final_answers(prediction)


12/13/2020 05:15:37 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.
Creating Embeddings: 100%|██████████| 1/1 [00:00<00:00, 62.77 Batches/s]
12/13/2020 05:15:37 - INFO - haystack.finder -   Got 10 candidates from retriever
12/13/2020 05:15:37 - INFO - haystack.finder -   Reader is looking for detailed answer in 174021 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  5.10 Batches/s]


In [6]:
# Run this cell to install mysql packages
!pip install mysql-connector-python

     |████████████████████████████████| 18.0MB 209kB/s 


In [7]:
# Run this cell to import the necessary libraries for connecting to the sql database
import mysql.connector
from mysql.connector import Error

In [9]:
# Run this cell to connect to the database, retrieve a question, find an answer to that question and send it again to the database. Once you are done, just stop the execution of the cell
while (True):
  connection = mysql.connector.connect(host='remotemysql.com',database='E786ozefXp',user='E786ozefXp',password='AJVykSfVuB')
  cursor = connection.cursor()
  cursor.execute("SELECT * from requests")
  myresult = cursor.fetchone()
  connection.close()
  if (myresult):
    req_id=myresult[0]
    username=myresult[1]
    question_user=myresult[2]
    print(req_id)
    print(username)
    print(question_user)
    #sql1 = "DELETE FROM requests WHERE req_id = %s"
    #val1 = (req_id)
    connection = mysql.connector.connect(host='remotemysql.com',database='E786ozefXp',user='E786ozefXp',password='AJVykSfVuB')
    cursor = connection.cursor()
    cursor.execute("DELETE FROM requests WHERE req_id = %s ",(req_id,))
    connection.commit()
    connection.close()
    print('Success1!')
    predictions=finder.get_answers(question=question_user, top_k_retriever=10, top_k_reader=2)
    pred=print_final_answers(predictions)
    print('Success2')
    #sql2 = "UPDATE users SET question = %s, answer= %s WHERE username = %s"
    #val2 = (question_user, pred, username )
    connection = mysql.connector.connect(host='remotemysql.com',database='E786ozefXp',user='E786ozefXp',password='AJVykSfVuB')
    cursor = connection.cursor()
    cursor.execute("UPDATE users SET question = %s, answer = %s WHERE username = %s",(question_user,pred,username,))
    connection.commit()
    connection.close()
    print('Success3')
  else:
    pass


50
user1
What is Arthritis?


12/13/2020 08:23:34 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.
Creating Embeddings: 100%|██████████| 1/1 [00:00<00:00, 58.14 Batches/s]
12/13/2020 08:23:34 - INFO - haystack.finder -   Got 10 candidates from retriever
12/13/2020 08:23:34 - INFO - haystack.finder -   Reader is looking for detailed answer in 71839 chars ...


Success1!


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.96 Batches/s]


Success2
Success3
51
user1
How to get rid of acne?


...

12/13/2020 08:41:13 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.
Creating Embeddings: 100%|██████████| 1/1 [00:00<00:00, 61.35 Batches/s]
12/13/2020 08:41:13 - INFO - haystack.finder -   Got 10 candidates from retriever
12/13/2020 08:41:13 - INFO - haystack.finder -   Reader is looking for detailed answer in 87464 chars ...


Success1!


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 15.02 Batches/s]


Success2
Success3


KeyboardInterrupt: ignored